In [ ]:
#CODE QUI DONNE LE CHEMIN ET LA LONGUEUR ENTRE DEUX MOTS


import xml.etree.ElementTree as ET
import networkx as nx

xml_file = r"C:\Users\jokle\Downloads\wolf.xml"
tree = ET.parse(xml_file) 
root = tree.getroot()

# Définition des mots à relier
word1 = "animal"
word2 = "chien"

synsets1 = get_synsets_for_word(word1)
synsets2 = get_synsets_for_word(word2)

# Trouver le plus court chemin et afficher le détail
if synsets1 and synsets2:
    try:
        shortest_path = nx.shortest_path(G, source=synsets1[0], target=synsets2[0])
        print(f" Chemin entre '{word1}' et '{word2}':\n")
        
        for synset_id in shortest_path:
            words = get_words_from_synset(synset_id)
            print(f" Synset {synset_id} → {', '.join(words) if words else 'Aucun mot'}")
        
    except nx.NetworkXNoPath:
        print(f" Aucun chemin trouvé entre '{word1}' et '{word2}'.")
else:
    print(" Un des mots n'existe pas dans WOLF.")


📏 Chemin entre 'animal' et 'chien':

🔹 Synset eng-30-00015388-n → animal, bête, créature, faune, être
🔹 Synset eng-30-01317541-n → animal domestique
🔹 Synset eng-30-02084071-n → canis familiaris, chien


In [88]:
#CODE QUI PASSE DE MOT A SYNSET ET INVERSE

def get_synsets_for_word(word):
    synsets = []
    for synset in root.findall(".//SYNSET"):
        for synonym in synset.findall(".//SYNONYM"):
            for literal in synonym.findall("LITERAL"):
                if literal.text and literal.text.lower() == word.lower():
                    synsets.append(synset.find("ID").text)
    return synsets

def get_words_from_synset(synset_id):
    words = []
    for synset in root.findall(".//SYNSET"):
        if synset.find("ID") is not None and synset.find("ID").text == synset_id:
            for synonym in synset.findall(".//SYNONYM"):
                for literal in synonym.findall("LITERAL"):
                    words.append(literal.text)
    return words

In [ ]:
#CREE LE GRAPHE

G = nx.DiGraph()

for synset in root.findall(".//SYNSET"):
    synset_id = synset.find("ID").text
    for ilr in synset.findall(".//ILR"):
        if ilr.get("type") == "hypernym":  # Utiliser uniquement les relations hypernymes
            G.add_edge(synset_id, ilr.text)

# Probleme de cycle : le code tourne à l'infini 
if not nx.is_directed_acyclic_graph(G):
    print(" Le graphe contient des cycles")
    G = nx.DiGraph(nx.DiGraph(G).reverse())  # Essayer d'inverser certaines relations
    G = nx.DiGraph([(u, v) for u, v in nx.edge_dfs(G) if u != v])  # Supprimer les boucles


In [ ]:
#CODE QUI DONNE LA HAUTEUR DU GRAPHE A PARTIR D'UN MOT

def get_tree_height_from_synset(G, synset_id):
    # On vérifie si le synset existe dans le graphe
    if synset_id not in G.nodes:
        print(f" Le synset {synset_id} n'existe pas dans le graphe.")
        return None
    
    # Effectue une recherche de la profondeur maximale
    longest_path_length = 0
    for node in G.nodes:
        if nx.has_path(G, synset_id, node):  # Si un chemin existe entre le synset et ce nœud
            path_length = nx.shortest_path_length(G, synset_id, node)
            longest_path_length = max(longest_path_length, path_length)
    
    return longest_path_length

#utilisation
synset_entite = get_synset_for_word('étalon')[0]  
height = get_tree_height_from_synset(G, synset_entite)

if height is not None:
    print(f" La hauteur de l'arbre à partir de {synset_entite} (entité) est : {height} niveaux.")



⚠️ Le graphe contient des cycles, tentative de correction...
🌲 La hauteur de l'arbre à partir de eng-30-02377181-n (entité) est : 1 niveaux.
